In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)  

In [3]:
df = con.execute("""
            SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
            FROM bronze_z0019
            WHERE data_ingestao >= '2025-01-11'
            """).fetchdf()


In [4]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao','row'])
df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WERKS": "id_categoria"})
df_final = df_final.rename(columns={"MAINS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})

df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,1003,PREGO,BT10,100,50
2,10002,MARTELO,BT50,100,1500


In [5]:
df2 = df_final
df2 = df2.astype(
    {
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': str,
        'vl_preco': float
        }
    )

df2.dtypes
#df2.head(10)

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor     object
vl_preco         float64
dtype: object

In [12]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
                id BIGINT,
                nm_produto TEXT,
                id_categoria TEXT,
                id_fornecedor BIGINT,
                vl_preco FLOAT
            )
            """)
df2.dtypes
#df2.head(10)

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor     object
vl_preco         float64
dtype: object

In [14]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,1003,PREGO,BT10,100,50.0
2,10002,MARTELO,BT50,100,1500.0


In [15]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [40]:
df_resultado = con.execute("select *from produtos").fetchdf()
df_resultado.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,1003,PREGO,BT10,100,50.0
2,10002,MARTELO,BT50,100,1500.0
3,10001,PARAFUSO,BT10,100,100.0
4,1003,PREGO,BT10,100,50.0


In [39]:
con.execute("SELECT * FROM produtos WHERE nm_produto = 'MARTELO'").fetchdf()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10002,MARTELO,BT50,100,1500.0
1,10002,MARTELO,BT50,100,1500.0


In [41]:
con.close()